# 0.0. IMPORTS

In [1]:
import math
import pandas.io.sql as psql
import psycopg2 as pg
#import requests
#import pickle
import numpy   as np
import pandas  as pd
#import inflection
import seaborn as sns
#import xgboost as xgb
#import random
import warnings
warnings.filterwarnings( 'ignore' )

#from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
#from tabulate              import tabulate
from scipy                 import stats  as ss
from matplotlib            import pyplot as plt
from IPython.display       import Image
from IPython.core.display  import HTML
#from boruta                import BorutaPy
#from sklearn.ensemble      import RandomForestRegressor
#from sklearn.metrics       import mean_absolute_error, mean_squared_error
#from sklearn.linear_model  import LinearRegression, Lasso
#from flask                 import Flask, request, Response


## 0.1. Helper Functions

In [2]:
#def cross_validation( x_training, kfold, model_name, model, verbose = False ) :
#    mae_list = []
#    mape_list = []
#    rmse_list = []
#    
#    
#    for k in reversed( range( 1, kfold+1 ) ):
#        if verbose:
#            print('\nKFold Number: {}'.format (k))
#        # Start and End date for validation
#        validation_start_date = x_training['date'].max() - datetime.timedelta( days=k*6*7)
#        validation_end_date = x_training['date'].max() - datetime.timedelta( days=(k-1)*6*7)
#        
#        # Filtering Dataset
#        training = x_training[x_training['date'] < validation_start_date]
#        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]
#        
#        # training and validation dataset
#        # training
#        xtraining = training.drop(['date','sales'], axis = 1)
#        ytraining = training['sales']
#        
#        # validation
#        xvalidation = validation.drop(['date','sales'], axis = 1)
#        yvalidation = validation['sales']
#        
#        
#        # Model
#        m = model.fit(xtraining, ytraining)
#        
#        
#        # Prediction
#        yhat = m.predict(xvalidation)
#        
#        
#        
#        # Performance
#        m_result = ml_error( model_name, np.expm1( yvalidation ), np.expm1( yhat ) )
#        
#        # store performance of each kfold iteration
#        mae_list.append (m_result['MAE'])
#        mape_list.append (m_result['MAPE'])
#        rmse_list.append (m_result['RMSE'])
#        
#    return pd.DataFrame({'Model Name': model_name,
#                         'MAE CV': np.round( np.mean( mae_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
#                         'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
#                         'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str ) }, index = [0] )

def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    

#def cramer_v( x, y):
#    cm = pd.crosstab( x, y ).values
#    n = cm.sum()
#    r,k = cm.shape
#    
#    chi2 = ss.chi2_contingency(cm)[0]
#    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
#    
#    kcorr = k - (k-1)**2/(n-1)
#    rcorr = r - (r-1)**2/(n-1)
#    
#    return np.sqrt( (chi2corr/n) / (min(kcorr-1, rcorr-1) ) )
#
#def mean_percentage_error( y,yhat ):
#    return np.mean( ( y-yhat ) / y  )
#
#def mean_absolute_percentage_error( y,yhat ):
#    return np.mean(np.abs( ( y-yhat )/y ) )
#
#
#def ml_error(model_name, y, yhat):
#    mae = mean_absolute_error(y,yhat)
#    mape = mean_absolute_percentage_error(y,yhat)
#    rmse = np.sqrt(mean_squared_error(y,yhat))
#    
#    return pd.DataFrame ({'Model Name': model_name,
#                          'MAE': mae,
#                          'MAPE': mape,
#                          'RMSE': rmse}, index = [0])
#

In [3]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2. Loading data

In [4]:
# Credentials
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com' 
port = 5432 
database = 'comunidadedsdb' 
user = 'member' 
password ='cdspa' 

In [5]:
# Connect with Postgres

conn = pg.connect( user=user,
                   password=password,
                   host=host,
                   port=port,
                   database=database)


### 0.2.1. Schema Query

In [15]:
# Cursor
cursor = conn.cursor()

# Query Schema
query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

cursor.execute( query_schema )

record = cursor.fetchall()
print( record )

# Close cursor
cursor.close()

# Close Connection
conn.close()

[('pg_toast',), ('pg_temp_1',), ('pg_toast_temp_1',), ('pg_catalog',), ('information_schema',), ('public',), ('pa004',), ('pg_temp_4',), ('pg_toast_temp_4',), ('pa005',), ('pg_temp_9',), ('pg_toast_temp_9',)]


### 0.2.2. Table Query

In [17]:
# Cursor
cursor = conn.cursor()

# Query Tables
query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname= 'pa004'
"""

cursor.execute( query_tables )

record = cursor.fetchall()
print( record )

# Close cursor
cursor.close()

# Close Connection
conn.close()

[('users',), ('vehicle',), ('insurance',)]


### 0.2.3. Collect Data using Psycopg2

In [ ]:
## Cursor
#cursor = conn.cursor()
#
## Query Tables
#query_tables_users = """
#    SELECT *
#    FROM pa004.users u
#    WHERE u.age > 44
#    LIMIT 10
#"""
#
#cursor.execute( query_tables_users )
#
#record = cursor.fetchall()
#print( record )
#
## Close cursor
#cursor.close()
#
## Close Connection
#conn.close()

In [ ]:
#pd.DataFrame( record )
#data.columns = ['']
#data.head()

### 0.2.4. Collect Data using Pandas

In [6]:
# Query Tables
query_tables = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle  v ON ( u.id = v.id )
                       LEFT JOIN pa004.insurance i ON ( u.id = i.id )
    WHERE u.age > 17
"""
df1 = pd.read_sql( query_tables, conn )

In [7]:
df2 = df1.copy()

In [8]:
df2.head()

,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,7,Male,23,11.0,152.0,7,1,< 1 Year,Yes,7,0,23367.0,249,0
1,13,Female,41,15.0,14.0,13,1,1-2 Year,No,13,1,31409.0,221,0
2,18,Female,25,35.0,152.0,18,1,< 1 Year,No,18,1,46622.0,299,0
3,31,Female,26,8.0,160.0,31,1,< 1 Year,No,31,0,2630.0,136,0
4,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0


In [11]:
df3 = df2.drop( df2.columns[[5,9]], axis = 1).reset_index()
df3 = df3.rename(columns={'index': 'id'})

df3.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,0,Male,23,11.0,152.0,1,< 1 Year,Yes,0,23367.0,249,0
1,1,Female,41,15.0,14.0,1,1-2 Year,No,1,31409.0,221,0
2,2,Female,25,35.0,152.0,1,< 1 Year,No,1,46622.0,299,0
3,3,Female,26,8.0,160.0,1,< 1 Year,No,0,2630.0,136,0
4,4,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0


In [12]:
df_raw = df3.copy()

In [19]:
df_raw.to_csv('C:/Users/leonardo.faria/repos/health-insurance-by-leofaria/datasets/insurance_all_database.csv', index=False)

In [45]:
# Close Connection
conn.close()